In [11]:
import os,sys
import numpy as np
import json
import pickle
import pandas as pd

import mwparserfromhell as mwph
from utils import wtpGetLinkAnchor
from utils import get_feature_set
from utils import getLinks, normalise_title, normalise_anchor, ngram_iterator

In [35]:
wiki_id = "dewiki"

In [36]:
## get anchor dict and page-ids
anchors = pickle.load(open("../../data/{0}/{0}.anchors.pkl".format(wiki_id), "rb"))
pageids = pickle.load(open("../../data/{0}/{0}.pageids.pkl".format(wiki_id), "rb"))
redirects = pickle.load(open("../../data/{0}/{0}.redirects.pkl".format(wiki_id), "rb"))


In [37]:
N_pages = len(pageids)
print(N_pages)

2464154


In [38]:
N_anchors = len(anchors)
print(N_anchors)
N_links = 0
N_no_alt = 0
N_candidates = 0
for anchor, links in anchors.items():
    for l,n in links.items():
        N_links+=n
    N_candidates += len(links)
    if len(links)==1:
        N_no_alt += 1
print(N_links)
N_candidates_avg = N_candidates/N_anchors
print(N_candidates_avg)

2902724
61799422
1.3611738491155205


In [39]:
N_no_alt/N_anchors

0.8564441538361898

In [15]:
## training sentences
## load the training
train_set = []
N_train = 0
N_train_links = 0
with open("../../data/{0}/training/sentences_train.csv".format(wiki_id)) as fin:
    for line in fin:
        try:
            title, sent = line.split("\t")
            train_set.append((title, sent))
            inp_pairs = getLinks(sent, redirects=redirects, pageids=pageids)
            N_train += 1
            N_train_links += len(inp_pairs)
        except:
            continue


In [16]:
print(N_train)
print(N_train_links)

99993
301365


In [17]:
## load the training
test_set = []
N_test = 0
N_test_links = 0
with open("../../data/{0}/testing/sentences_test.csv".format(wiki_id)) as fin:
    for line in fin:
        try:
            title, sent = line.split("\t")
            test_set.append((title, sent))
            inp_pairs = getLinks(sent, redirects=redirects, pageids=pageids)
            N_test += 1
            N_test_links += len(inp_pairs)
        except:
            continue

In [18]:
print(N_test)
print(N_test_links)

99987
302278


In [19]:
dict_pd = {}

In [27]:

list_N_pages = []
list_N_anchors = []
list_N_links = []
list_N_cand_avg = []
list_N_train = []
list_N_train_links = []
list_N_test = []
list_N_test_links = []



list_wiki_id = ["arwiki","bnwiki","cswiki","viwiki","dewiki","ptwiki","simplewiki"]
for wiki_id in list_wiki_id:
# wiki_id = "arwiki"
    print(wiki_id)
    ## get anchor dict and page-ids
    anchors = pickle.load(open("../../data/{0}/{0}.anchors.pkl".format(wiki_id), "rb"))
    pageids = pickle.load(open("../../data/{0}/{0}.pageids.pkl".format(wiki_id), "rb"))
    redirects = pickle.load(open("../../data/{0}/{0}.redirects.pkl".format(wiki_id), "rb"))

    N_pages = len(pageids)
    N_anchors = len(anchors)
    N_links = 0
    N_candidates = 0
    for anchor, links in anchors.items():
        for l,n in links.items():
            N_links+=n
        N_candidates += len(links)
    N_candidates_avg = N_candidates/N_anchors

    train_set = []
    N_train = 0
    N_train_links = 0
    with open("../../data/{0}/training/sentences_train.csv".format(wiki_id)) as fin:
        for line in fin:
            try:
                title, sent = line.split("\t")
                train_set.append((title, sent))
                inp_pairs = getLinks(sent, redirects=redirects, pageids=pageids)
                N_train += 1
                N_train_links += len(inp_pairs)
            except:
                continue
    ## load the training
    test_set = []
    N_test = 0
    N_test_links = 0
    with open("../../data/{0}/testing/sentences_test.csv".format(wiki_id)) as fin:
        for line in fin:
            try:
                title, sent = line.split("\t")
                test_set.append((title, sent))
                inp_pairs = getLinks(sent, redirects=redirects, pageids=pageids)
                N_test += 1
                N_test_links += len(inp_pairs)
            except:
                continue
            
    list_N_pages += [int(N_pages)]
    list_N_anchors += [int(N_anchors)]
    list_N_links += [int(N_links)]
    list_N_cand_avg += [np.round(N_candidates_avg,2)]
    list_N_train += [int(N_train)]
    list_N_train_links += [int(N_train_links)]
    list_N_test += [int(N_test)]
    list_N_test_links += [int(N_test_links)]


arwiki
bnwiki
cswiki
viwiki
dewiki
ptwiki
simplewiki


In [45]:
list_index=list_wiki_id
list_columns = ['#pages',  '#links','#anchors', '#train sent.', '#train links', '#test sent.', '#test links']

df = pd.DataFrame(columns=list_columns, index=list_index)
df['#pages'] = [ "{:,}".format(int(h)) for h in  list_N_pages]
df['#links'] = [ "{:,}".format(int(h)) for h in list_N_links]
df['#anchors'] = [ "{:,}".format(int(h)) for h in list_N_anchors]
# df['N cand. avg.'] = list_N_cand_avg
df['#train sent.'] = [ "{:,}".format(int(h)) for h in  list_N_train]
df['#train links'] = [ "{:,}".format(int(h)) for h in  list_N_train_links]
df['#test sent.'] = [ "{:,}".format(int(h)) for h in list_N_test]
df['#test links'] = [ "{:,}".format(int(h)) for h in list_N_test_links]
df

,#pages,#links,#anchors,#train sent.,#train links,#test sent.,#test links
arwiki,"1,057,579","11,346,322","562,058","99,993","301,365","99,987","302,278"
bnwiki,"92,401","1,120,949","97,739","43,678","77,889","43,679","78,433"
cswiki,"459,769","10,151,007","648,544","99,998","232,489","99,997","232,895"
viwiki,"1,251,872","12,133,339","347,514","99,999","208,527","100,000","208,298"
dewiki,"2,464,154","61,799,422","2,902,724","99,998","240,285","99,999","240,646"
ptwiki,"1,041,138","20,428,721","951,155","99,999","362,608","99,995","360,799"
simplewiki,"169,558","2,124,488","137,215","82,089","182,225","82,091","182,644"


In [30]:
len(anchors)

137215

In [31]:
N_anchors = len(anchors)
print(N_anchors)
N_links = 0

N_candidates = 0
N_no_alt = 0
for anchor, links in anchors.items():
    for l,n in links.items():
        N_links+=n
    N_candidates += len(links)
    if len(links)==1:
        N_no_alt += 1
print(N_links)
N_candidates_avg = N_candidates/N_anchors
print(N_candidates_avg)

137215
2124488
1.2095689246802463


In [34]:
N_no_alt/N_anchors

0.8891229093029188